## Data augmentation

Our dataset is very badly equilibrated. The goal is to augmente the data with have in a few less-representated grades.

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("..//datasets//avisassurance_train.csv", encoding="utf-8", sep = ";")
train

,date,note,auteur,avis,assureur,produit
0,06 septem...,5,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto
1,03 mai 20...,4,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto
2,21 mars 2...,5,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto
3,10 juin 2...,4,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto
4,29 janvie...,1,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto
...,...,...,...,...,...,...
24100,22 mars 2...,1,hophop-107522,Assurance moto chez la mutuel des motards en F...,Mutuelle des Motards,moto
24101,06 décemb...,1,tzl-81680,Même les demandes les plus simples n'aboutisse...,Allianz,habitation
24102,14 avril ...,1,jmr-72500-110395,"En décembre 2019, j'ai souscrit à un contrat C...",Cegema Assurances,sante
24103,11 juille...,3,cris-77532,Je suis assurer à la gmf depuis plus de 15 ans...,GMF,auto


In [8]:
import gensim.downloader as api

In [9]:
twitter_25_model = api.load('glove-twitter-25')  
twitter_25_model.most_similar('awesome', topn=5)

[('amazing', 0.9687871932983398),
 ('best', 0.9600659608840942),
 ('fun', 0.9331520795822144),
 ('fantastic', 0.9313924312591553),
 ('perfect', 0.92434161901474)]

In [104]:
twitter_25_model.most_similar('i')[0][0]

"'m"

## Plan

##### 1- Traduction from french to english
##### 2- Replacing a few words from the review with most similar one (using gensim most_similar)
##### 3- Traduction from english to french
##### 4- Adding data with the new review to our dataset

----------------------------
##### 1- Traduction from french to english

In [82]:
from googletrans import Translator
translator = Translator()

def translation_dest_src(review: str, source: str = 'fr', destination: str = 'en') -> str:
    return translator.translate(review, src = source, dest = destination).text

In [99]:
test_original = "L'assurance AXA est très mauvais en assurance santé. Les indemnisations sont très lentes. Balladé de médecin en médecin (expert, arbitre,...), tout est prévu pour perdre du temps. Très mauvaise assurance en santé."
test_en = translation_dest_src(test_original,
                source = 'fr',
                destination = 'en')
test_en

'AXA insurance is very bad in health insurance.Compensation is very slow.Balladed doctor in a doctor (expert, referee, ...), everything is planned to waste time.Very poor health insurance.'

--------------------------------
##### 2- Replacing a few words from the review with most similar one (using gensim most_similar)

The idea is to pick random words in our review and replace them with similar others. We will try with a different ratio replacement to see how it reacts.

In [106]:
from random import choice

def get_most_similar_word(word: str) -> str:
    # Trying to get the most similar word
    try:
        return twitter_25_model.most_similar(word.lower())[0][0]
    # If not returning the original word
    except:
        print("Couldn't find the most similar word.")
        return word

def word_replacement(review_eng: str, replacement_ratio: float = 0.5) -> str:
    # Splitting the review by words and counting the number of words
    review_eng_words = review_eng.split()
    nb_words = len(review_eng_words)

    # Counting the number of words to replace
    nb_words_to_change = int(nb_words*replacement_ratio)

    # Creating a list with the placement of changed words, so we don't change twice the same word (could result as getting back to original)
    word_placement_changed = list()
    for words in range(nb_words_to_change):
        # Getting a random int as the word to change
        word_placement = choice([_ for _ in range(nb_words) if _ not in word_placement_changed])

        # Replacing the original word with the most similar one
        review_eng_words[word_placement] = get_most_similar_word(review_eng_words[word_placement])

        # Appending the word placement in our list to avoid changing twice the same
        word_placement_changed.append(word_placement)
        # print(' '.join(review_eng_words))

    return (' '.join(review_eng_words))

In [107]:
test_replaced = word_replacement(test_en, 0.5)
test_replaced

Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.


'AXA insurance is such bad over health insurance.Compensation was such slow.Balladed doc in no doc (expert, referee, ...), something was planned to waste time.Very poor education insurance.'

--------------------------------
##### 3- Traduction from english to french

In [97]:
test_replaced = translation_dest_src(test_replaced, source= 'en', destination= 'fr')
test_replaced

"AXA ASSURANCE Était une telle merde sur l'assurance maladie.Les pypencations étaient si lentes. Médecin Ballonnier dans une DOC (expert, arbitre, ...), quelque chose est prévu pour le temps perdu. Très mauvaise assurance maladie."

In [101]:
test_original

"L'assurance AXA est très mauvais en assurance santé. Les indemnisations sont très lentes. Balladé de médecin en médecin (expert, arbitre,...), tout est prévu pour perdre du temps. Très mauvaise assurance en santé."

--------------------------------
##### 4- Adding data with the new review to our dataset